In [84]:
import torch
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel
import os
from torcheval.metrics import BinaryAccuracy, BinaryF1Score, BinaryConfusionMatrix, BinaryPrecisionRecallCurve

def clip_pred(imgs):
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    inputs = processor(
        text=["a synthetic image created by AI", "a real image taken by a human"],
        images=imgs,
        return_tensors="pt",
        padding=True
    )

    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
    prob = logits_per_image.softmax(dim=1)  
    return prob

In [94]:
probs = clip_pred(images)
probs[:,1]

tensor([0.0036, 0.0354], grad_fn=<SelectBackward0>)

In [69]:
def load_images_from_folder(folder):
    images = []
    
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
        
    
    labels = torch.zeros(len(images), dtype=torch.int)
    return images, labels

In [70]:
def metrics(ys, ts):
    acc = BinaryAccuracy()
    f1 = BinaryF1Score()
    cm = BinaryConfusionMatrix()
    acc.update(ys, ts)
    f1.update(ys, ts)
    cm.update(ys, ts)
    return acc.compute(), f1.compute(), cm.compute()


In [98]:
folder_path = r'C:\Users\vikto\Documents\GitHub\DTU_repo\deep_learning\4_Convolutional\images'
images, labels = load_images_from_folder(folder_path)
probs = clip_pred(images)
metrics(probs[:,1], labels)

(tensor(1.),
 tensor(0.),
 tensor([[2., 0.],
         [0., 0.]]))

## With labels implemented

In [ ]:
import torch
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel
import os
from torcheval.metrics import BinaryAccuracy, BinaryF1Score, BinaryConfusionMatrix, BinaryPrecisionRecallCurve

def clip_pred(imgs, imgs_class):
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    logits = torch.empty(len(imgs),2)

    for i in range(len(imgs)):
        input = processor(
            text=["a synthetic image of a" + str(imgs_class[i]), "a real image of a" + str(imgs_class[i])],
            images=imgs[i],
            return_tensors="pt",
            padding=True
        )

        output = model(**input)
        output.logits_per_image  # this is the image-text similarity score
    prob = logits.softmax(dim=1)  
    return prob


def load_images_from_folder(folder):
    images = []
    class_types = {'(2)':'automobile','(3)': 'bird', '(4)': 'cat', '(5)': 'deer', '(6)': 'dog', '(7)': 'frog', '(8)': 'horse', '(9)': 'ship', '(10)': 'truck'}
    imgs_class = []

    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder, filename))
        if img is not None:
            images.append(img)

            for c in class_types.keys():
                if c in filename:
                    imgs_class.append(class_types.get(c))
                    break
                else:
                    imgs_class.append('airplane')
                    break

    labels = torch.zeros(len(images), dtype=torch.int)
    return images, labels, imgs_class

def metrics(ys, ts):
    acc = BinaryAccuracy()
    f1 = BinaryF1Score()
    cm = BinaryConfusionMatrix()
    prc = BinaryPrecisionRecallCurve()
    acc.update(ys, ts)
    f1.update(ys, ts)
    cm.update(ys, ts)
    prc.update(ys, ts)

    return acc.compute(), f1.compute(), cm.compute(), prc.compute()


folder_path = r'C:\Users\vikto\Documents\GitHub\DTU_repo\deep_learning\4_Convolutional\images'
images, labels, imgs_class = load_images_from_folder(folder_path)
probs = clip_pred(images, imgs_class)
metrics(probs[:,1], labels)

c:\Users\vikto\miniconda3\envs\deep_learning\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


TypeError: cat() received an invalid combination of arguments - got (Tensor, Tensor, dim=int), but expected one of:
 * (tuple of Tensors tensors, int dim, *, Tensor out)
 * (tuple of Tensors tensors, name dim, *, Tensor out)


In [66]:
t =torch.empty(100,2)
ps = torch.tensor([0.01,0.99])
t_new = torch.cat(t,ps)



TypeError: cat() received an invalid combination of arguments - got (Tensor, Tensor), but expected one of:
 * (tuple of Tensors tensors, int dim, *, Tensor out)
 * (tuple of Tensors tensors, name dim, *, Tensor out)
